### 검색 기법 개요

| 기법 | 핵심 원리 | 장점 | 적합한 상황 |
| :--- | :--- | :--- | :--- |
| Vector Store Retriever | 벡터 유사도 검색 | 간단, 빠름 | 기본 RAG 구현 |
| Multi Query Retriever | 다중 쿼리 생성 | 검색 범위 확장 | 모호한 질문 처리 |
| Contextual Compression | 관련 내용만 추출 | 노이즈 제거, 비용 절감 | 긴 문서에서 핵심 추출 |
| Ensemble Retriever | BM25 + 벡터 결합 | 키워드와 의미 검색 통합 | 전문 용어가 중요한 도메인 |
| Decomposition | 질문 분해 | 복잡한 질문 처리 | 다단계 추론 필요 |
| Step Back | 추상화 질문 생성 | 배경 지식 활용 | 구체적 세부사항 질문 |
| HyDE | 가상 문서 임베딩 | 질문-문서 갭 해소 | 짧은 질문, 긴 문서 |

### 선택 가이드

- Vector Store Retriever: 모든 RAG 시스템의 기본. 단순하고 빠른 검색이 필요할 때
- Multi Query Retriever: 사용자 질문이 다양하게 해석될 수 있을 때
- Contextual Compression: 검색된 문서가 길고 관련 없는 내용이 많을 때

- Ensemble Retriever: 키워드 매칭이 중요한 기술/법률/의료 도메인
- RAG-Fusion: 검색 결과의 다양성과 정확도를 모두 높이고 싶을 때
- Decomposition: "A와 B의 차이점은?" 같은 복합 질문 처리
- Step Back: "2023년 삼성전자 반도체 매출은?" 같이 구체적 사실 질문
- HyDE: 짧은 질문으로 긴 문서를 검색할 때 (질문-문서 임베딩 불일치 해소)

### 성능 비교
| 기법 | 검색 정확도 | 지연 시간 | LLM 호출 | 복잡도 |
| :--- | :--- | :--- | :--- | :--- |
| Vector Store | 중간 | 낮음 | 0 | 낮음 |
| Multi Query | 높음 | 중간 | 1 | 중간 |
| Contextual Compression | 높음 | 높음 | N | 중간 |
| Ensemble | 높음 | 중간 | 0 | 중간 |
| RAG-Fusion | 매우 높음 | 높음 | 1+ | 높음 |
| Decomposition | 매우 높음 | 매우 높음 | 2+ | 높음 |
| Step Back | 높음 | 높음 | 1 | 중간 |
| HyDE | 높음 | 중간 | 1 | 중간 |

### Vector Store Retriver

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

loader = PyMuPDFLoader(r'E:\Portfolio\Study\hello-langchain\rag_data\300720_한일시멘트_2023.pdf')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

228

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = FAISS.from_documents(documents=documents,
                                   embedding=embeddings_model,
                                   distance_strategy=DistanceStrategy.COSINE)

In [10]:
query = "한일시멘트의 주요 approach를 알려줘"

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50}
)

docs = retriever.invoke(query)
print(len(docs))
docs[0]


5


Document(id='89a9dff4-dad2-4a1e-87df-e3bef9d4ece4', metadata={'producer': 'iLovePDF', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2023-07-28T10:47:36+09:00', 'source': 'E:\\Portfolio\\Study\\hello-langchain\\rag_data\\300720_한일시멘트_2023.pdf', 'file_path': 'E:\\Portfolio\\Study\\hello-langchain\\rag_data\\300720_한일시멘트_2023.pdf', 'total_pages': 79, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-07-28T11:18:40+09:00', 'trapped': '', 'modDate': "D:20230728111840+09'00'", 'creationDate': "D:20230728104736+09'00'", 'page': 16}, page_content='된 핵심이슈는 기후변화 대응 전략, 오염물질 관리, 안전하고 건강한 작업환경 구축, 친환경 사업 개\n발, 소비자 만족 및 제품 품질이며, 본 보고서를 통해 이를 중심으로 하는 주요 활동 성과와 향후 계\n획을 투명하게 보고하고자 합니다.\nSTEP 3\n영향의 중대성 평가\nSTEP 4\n우선순위 유효성 검토 및  \n핵심 이슈 도출\n한일의 경영활동 및 비즈니스 관계를 정리하고 지속가능경영에 영향을 미치는 이슈의 원인관계를 \n파악합니다. 또한 이해관계자 집단의 정의와 식별 기준 마련 등으로 한일을 둘러싼 ESG경영 환경을 \n이해하고 구체적인 정의를 마련합니다.  \n한일은 이해관계자와 한일에게 영향을 미치는 ESG이슈와 영향정도를 파악하여 효율적인 ESG경영\n을 추진하고자  중대성평

In [13]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = init_chat_model(model="gpt-4o-mini", temperature=0, max_tokens=500)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({'context': (format_docs(docs)), 'question':query})
response

'한일시멘트의 주요 approach는 다음과 같습니다:\n\n1. **기후변화 대응 전략**: 기후변화에 효과적으로 대응하기 위한 전략을 수립하고 실행합니다.\n2. **오염물질 관리**: 오염물질을 체계적으로 관리하여 환경 보호에 기여합니다.\n3. **안전하고 건강한 작업환경 구축**: 직원들이 안전하고 건강하게 일할 수 있는 환경을 조성합니다.\n4. **친환경 사업 개발**: 지속 가능한 친환경 사업을 개발하여 환경 영향을 최소화합니다.\n5. **소비자 만족 및 제품 품질**: 소비자의 요구를 충족시키고 제품의 품질을 지속적으로 향상시키기 위해 노력합니다.\n\n이러한 approach는 한일의 지속가능경영 전략에 반영되어 있으며, 이해관계자와의 소통을 강화하고 ESG 경영을 효율적으로 추진하기 위한 중대성 평가 프로세스를 통해 지속적으로 개선되고 있습니다.'

### Multi Query Retriever

In [ ]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

llm = init_chat_model(model="gpt-4o-mini", temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

unique_docs = retriever_from_llm.invoke(query)
print(f"검색된 문서 수: {len(unique_docs)}")


검색된 문서 수: 7


In [15]:
unique_docs[0]

Document(id='19b52f2d-481d-4382-8c2a-6b8393196e10', metadata={'producer': 'iLovePDF', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2023-07-28T10:47:36+09:00', 'source': 'E:\\Portfolio\\Study\\hello-langchain\\rag_data\\300720_한일시멘트_2023.pdf', 'file_path': 'E:\\Portfolio\\Study\\hello-langchain\\rag_data\\300720_한일시멘트_2023.pdf', 'total_pages': 79, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-07-28T11:18:40+09:00', 'trapped': '', 'modDate': "D:20230728111840+09'00'", 'creationDate': "D:20230728104736+09'00'", 'page': 4}, page_content='십을 구축하여 사회적 가치 창출에도 기여하겠습니다.\n이해관계자 여러분께 한일의 지속가능경영에 대한 의지와 추진 과정을 알리고 이해\n를 돕기 위해 최초로 그룹 관점의 지속가능경영보고서를 발간하게 되었습니다. 지속\n가능한 가치를 창출하는 한일의 긴 여정에 이해관계자 여러분들의 변함없는 기대와 \n관심을 부탁드립니다. \n감사합니다.\n한일홀딩스   \n대표이사  박지훈\n5\nHANIL Sustainability Report\n4\nHANIL Sustainability Report')

In [17]:
from langchain_core.runnables import RunnablePassthrough
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke('한일시멘트의 주요 approach를 요약해서 알려주세요.')
print(response)

한일시멘트의 주요 접근 방식은 지속가능한 경영을 통해 사회적 가치를 창출하고, 기후변화 대응, 오염물질 관리, 안전하고 건강한 작업환경 구축, 친환경 사업 개발, 소비자 만족 및 제품 품질을 중심으로 한 핵심 이슈를 관리하는 것입니다. 이를 위해 한일은 ESG(환경, 사회, 지배구조) 경영을 강화하고, 이해관계자와의 소통을 통해 지속가능한 발전 목표(UN SDGs)와 연계된 활동을 추진하고 있습니다. 또한, 중대성 평가를 통해 우선적으로 대응해야 할 이슈를 도출하고, 이를 기반으로 구체적인 전략과 KPI를 설정하여 성과를 관리하고 있습니다.
